In [1]:
%matplotlib inline
from skimage import data, io, filters, exposure
from skimage.segmentation import slic, felzenszwalb
from skimage.transform import resize
from skimage.morphology import square, closing, binary_closing
from matplotlib import pyplot as plt
from skimage import color
import numpy as np
from scipy import ndimage as ndi
from skimage.measure import moments_hu, moments, moments_central, moments_normalized

In [2]:
def get_feature(name):
    sample = io.imread(str(name))
    sample = resize(sample, (148,224), mode='constant')

    sample_hsv = color.convert_colorspace(sample,'RGB','HSV')
    
    # change green color to white, other color to black
    for i in range(sample_hsv.shape[0]):
        for j in range(sample_hsv.shape[1]):
            if sample_hsv[i,j,0] > 0.2083 and sample_hsv[i,j,0] < 0.4167 and sample_hsv[i,j,2]>0.1  and sample_hsv[i,j,1]>0.1:
                sample_hsv[i,j,1] = 0
                sample_hsv[i,j,2] = 1
            else:
                sample_hsv[i,j,2] = 0

    sample_rgb = color.convert_colorspace(sample_hsv,'HSV','RGB')

    sample_grey = color.rgb2grey(sample_rgb)

    # find center of the image
    t = 0
    res_i = 0
    res_j = 0
    for i in range(sample_grey.shape[0]-100):
        for j in range(sample_grey.shape[1]-100):
            if np.sum(sample_grey[i:i+100,j:j+100]) > t:
                res_i = i
                res_j = j
                t = np.sum(sample_grey[i:i+100,j:j+100])

    sample_cropped = sample_grey[res_i:res_i+100,res_j:res_j+100]

    closed = binary_closing(sample_cropped, square(2))

    closed = np.asarray(closed, dtype=np.double)

    M = moments(closed)
    cr = M[1, 0] / M[0, 0]
    cc = M[0, 1] / M[0, 0]

    mu = moments_central(closed, center=(cr,cc))
    nu = moments_normalized(mu)
    return moments_hu(nu)

In [3]:
feature = np.empty((0,8), int)

from os import listdir
from os.path import isfile, join
mypath = 'train_1'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for o in onlyfiles:
    feature = np.vstack((feature, np.hstack((get_feature(join(mypath,o)),0))))

In [4]:
mypath = 'train_2'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for o in onlyfiles:
    feature = np.vstack((feature,np.hstack((get_feature(join(mypath,o)),1))))

In [5]:
feature_test = np.empty((0,8), int)
mypath = 'test'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

x = 0
for o in onlyfiles:
    x = int(o[8:10])
    if x<67:
        u=0
    else:
        u=1
    feature_test = np.vstack((feature_test,np.hstack((get_feature(join(mypath,o)),u))))
    x += 1
feature_test.shape

(21, 8)

In [6]:
from sklearn.neural_network import MLPClassifier
# 15 0.6 50
# 9-3, 200, 0.1, 1500
mlp = MLPClassifier(hidden_layer_sizes=(9,3),
                    solver='sgd',
                    batch_size=200,
                    activation='relu',
                    learning_rate='adaptive',
                    learning_rate_init=0.1,
                    max_iter=1500,
                    momentum=0,
                    tol=0.0,
                    alpha=0,
                    verbose=True,
                    random_state=123)
train_x = feature[:,:-1]
train_y = feature[:,-1]
test_x = feature_test[:,:-1]
test_y = feature_test[:,-1]
test_x.shape, test_y.shape

mlp.fit(train_x, train_y)
print(mlp.score(test_x,test_y))
print(mlp.predict(test_x))

/home/rian/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:351: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn("Got `batch_size` less than 1 or larger than "


Iteration 1, loss = 0.70239386
Iteration 2, loss = 0.70062115
Iteration 3, loss = 0.69908110
Iteration 4, loss = 0.69774520
Iteration 5, loss = 0.69658813
Iteration 6, loss = 0.69558743
Iteration 7, loss = 0.69472321
Iteration 8, loss = 0.69397789
Iteration 9, loss = 0.69333597
Iteration 10, loss = 0.69278380
Iteration 11, loss = 0.69230942
Iteration 12, loss = 0.69190233
Iteration 13, loss = 0.69155336
Iteration 14, loss = 0.69125451
Iteration 15, loss = 0.69099882
Iteration 16, loss = 0.69078025
Iteration 17, loss = 0.69059356
Iteration 18, loss = 0.69043420
Iteration 19, loss = 0.69029826
Iteration 20, loss = 0.69018237
Iteration 21, loss = 0.69008361
Iteration 22, loss = 0.68999948
Iteration 23, loss = 0.68992783
Iteration 24, loss = 0.68986682
Iteration 25, loss = 0.68981488
Iteration 26, loss = 0.68977066
Iteration 27, loss = 0.68973300
Iteration 28, loss = 0.68970091
Iteration 29, loss = 0.68967356
Iteration 30, loss = 0.68965024
Iteration 31, loss = 0.68963033
Iteration 32, los

Iteration 362, loss = 0.68912296
Iteration 363, loss = 0.68912156
Iteration 364, loss = 0.68912016
Iteration 365, loss = 0.68911876
Iteration 366, loss = 0.68911735
Iteration 367, loss = 0.68911595
Iteration 368, loss = 0.68911454
Iteration 369, loss = 0.68911313
Iteration 370, loss = 0.68911172
Iteration 371, loss = 0.68911031
Iteration 372, loss = 0.68910890
Iteration 373, loss = 0.68910749
Iteration 374, loss = 0.68910607
Iteration 375, loss = 0.68910466
Iteration 376, loss = 0.68910324
Iteration 377, loss = 0.68910182
Iteration 378, loss = 0.68910040
Iteration 379, loss = 0.68909898
Iteration 380, loss = 0.68909756
Iteration 381, loss = 0.68909614
Iteration 382, loss = 0.68909472
Iteration 383, loss = 0.68909329
Iteration 384, loss = 0.68909186
Iteration 385, loss = 0.68909044
Iteration 386, loss = 0.68908901
Iteration 387, loss = 0.68908758
Iteration 388, loss = 0.68908615
Iteration 389, loss = 0.68908471
Iteration 390, loss = 0.68908328
Iteration 391, loss = 0.68908184
Iteration 

Iteration 664, loss = 0.68863729
Iteration 665, loss = 0.68863544
Iteration 666, loss = 0.68863360
Iteration 667, loss = 0.68863175
Iteration 668, loss = 0.68862990
Iteration 669, loss = 0.68862805
Iteration 670, loss = 0.68862620
Iteration 671, loss = 0.68862434
Iteration 672, loss = 0.68862248
Iteration 673, loss = 0.68862063
Iteration 674, loss = 0.68861877
Iteration 675, loss = 0.68861690
Iteration 676, loss = 0.68861504
Iteration 677, loss = 0.68861318
Iteration 678, loss = 0.68861131
Iteration 679, loss = 0.68860944
Iteration 680, loss = 0.68860757
Iteration 681, loss = 0.68860570
Iteration 682, loss = 0.68860382
Iteration 683, loss = 0.68860195
Iteration 684, loss = 0.68860007
Iteration 685, loss = 0.68859819
Iteration 686, loss = 0.68859631
Iteration 687, loss = 0.68859442
Iteration 688, loss = 0.68859254
Iteration 689, loss = 0.68859065
Iteration 690, loss = 0.68858876
Iteration 691, loss = 0.68858687
Iteration 692, loss = 0.68858498
Iteration 693, loss = 0.68858309
Iteration 

Iteration 1034, loss = 0.68781401
Iteration 1035, loss = 0.68781134
Iteration 1036, loss = 0.68780866
Iteration 1037, loss = 0.68780598
Iteration 1038, loss = 0.68780330
Iteration 1039, loss = 0.68780061
Iteration 1040, loss = 0.68779793
Iteration 1041, loss = 0.68779523
Iteration 1042, loss = 0.68779254
Iteration 1043, loss = 0.68778984
Iteration 1044, loss = 0.68778714
Iteration 1045, loss = 0.68778444
Iteration 1046, loss = 0.68778174
Iteration 1047, loss = 0.68777903
Iteration 1048, loss = 0.68777632
Iteration 1049, loss = 0.68777360
Iteration 1050, loss = 0.68777089
Iteration 1051, loss = 0.68776817
Iteration 1052, loss = 0.68776544
Iteration 1053, loss = 0.68776272
Iteration 1054, loss = 0.68775999
Iteration 1055, loss = 0.68775726
Iteration 1056, loss = 0.68775452
Iteration 1057, loss = 0.68775179
Iteration 1058, loss = 0.68774905
Iteration 1059, loss = 0.68774630
Iteration 1060, loss = 0.68774356
Iteration 1061, loss = 0.68774081
Iteration 1062, loss = 0.68773806
Iteration 1063

Iteration 1438, loss = 0.68645429
Iteration 1439, loss = 0.68645009
Iteration 1440, loss = 0.68644588
Iteration 1441, loss = 0.68644166
Iteration 1442, loss = 0.68643744
Iteration 1443, loss = 0.68643322
Iteration 1444, loss = 0.68642899
Iteration 1445, loss = 0.68642476
Iteration 1446, loss = 0.68642052
Iteration 1447, loss = 0.68641627
Iteration 1448, loss = 0.68641202
Iteration 1449, loss = 0.68640777
Iteration 1450, loss = 0.68640351
Iteration 1451, loss = 0.68639924
Iteration 1452, loss = 0.68639497
Iteration 1453, loss = 0.68639070
Iteration 1454, loss = 0.68638642
Iteration 1455, loss = 0.68638213
Iteration 1456, loss = 0.68637784
Iteration 1457, loss = 0.68637355
Iteration 1458, loss = 0.68636925
Iteration 1459, loss = 0.68636494
Iteration 1460, loss = 0.68636063
Iteration 1461, loss = 0.68635632
Iteration 1462, loss = 0.68635200
Iteration 1463, loss = 0.68634767
Iteration 1464, loss = 0.68634334
Iteration 1465, loss = 0.68633900
Iteration 1466, loss = 0.68633466
Iteration 1467

/home/rian/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
